In [ ]:
import pandas as pd

# Import Databases

In [ ]:
nodes = pd.read_csv(r'..\HQ_DIR\graph_files\nodes_relation_4type.csv', header=None)
nodes[['DOID','Type']] = nodes[0].str.split('\t', expand=True)
nodes = nodes.drop(columns=[0])
nodes

In [ ]:
edges = pd.read_csv(r'..\HQ_DIR\graph_files\edges_relation_4type.csv', header=None)
edges[['node1', 'edge', 'node2', 'idk1','idk2']] = edges[0].str.split('\t', expand=True)
edges = edges.drop(columns=[0])
edges

In [ ]:
tn_edges = pd.read_csv(r'..\HQ_DIR\graph_files\TN_edges_relation_4type.csv', header=None)
tn_edges[['node1', 'edge', 'node2', 'idk1', 'idk2']] = tn_edges[0].str.split('\t', expand=True)
tn_edges = tn_edges.drop(columns=[0])
tn_edges

# Disease Pool + Gene2Disease

In [ ]:
dp = pd.read_csv(r'..\HQ_DIR\source\Disease_Pool.csv', dtype={'DOID': str})
disease_pool = ["DOID:" + str(doid) for doid in dp['DOID']]
print(disease_pool)
print(len(disease_pool))

In [ ]:
# Gene2Disease
gene_dis = edges[edges['edge'] == 'GENE_DIS']
Gene2Disease = gene_dis[gene_dis['node2'].isin(disease_pool)]
Gene2Disease

# Drug Pool + Drug2Gene

In [ ]:
drp = pd.read_csv(r'..\HQ_DIR\source\filtered_cid.csv')
drug_pool = drp['0'].tolist()
print(drug_pool)
print(len(drug_pool))

In [ ]:
# Drug2Gene
drug_gene_edge_types = [
    'GENE_DRUG',
    'DRUG_ACTIVATION_GENE',
    'DRUG_BINDING_GENE',
    'DRUG_CATALYSIS_GENE',
    'DRUG_EXPRESSION_GENE',
    'DRUG_INHIBITION_GENE',
    'DRUG_BINDACT_GENE',
    'DRUG_BINDINH_GENE',
    'DRUG_REACTION_GENE',
    'DRUG_PREDBIND_GENE'
]

drug_pool_str = ['PUBCHEM.COMPOUND:' + str(cid) for cid in drug_pool]


drug_gene = edges[edges['edge'].isin(drug_gene_edge_types)]
Drug2Gene = drug_gene[drug_gene['node1'].isin(drug_pool_str)]
Drug2Gene

# Disease2Drug

In [ ]:
tp_dis_drug = edges[edges['edge'] == 'DIS_DRUG']
Disease2Drug = tp_dis_drug[tp_dis_drug['node1'].isin(disease_pool)]
Disease2Drug = Disease2Drug[Disease2Drug['node2'].isin(drug_pool_str)]
Disease2Drug

# Gene2Gene

In [ ]:
gene_gene = edges[edges['edge'].isin([
    'GENE_GENE',
    'GENE_ACTIVATION_GENE',
    'GENE_BINDING_GENE',
    'GENE_CATALYSIS_GENE',
    'GENE_EXPRESSION_GENE',
    'GENE_INHIBITION_GENE',
    'GENE_BINDACT_GENE',
    'GENE_BINDINH_GENE',
    'GENE_REACTION_GENE',
    'GENE_PTMOD_GENE'
])]

gene_gene

# Disease Pool

In [ ]:
disease_pool = set(Disease2Drug['node1']).union(set(Gene2Disease['node2']))
print(disease_pool)
print(len(disease_pool))

# Drug Pool

In [ ]:
drug_pool = set(Drug2Gene['node1']).union(set(Disease2Drug['node2']))
print(drug_pool)
print(len(drug_pool))

In [ ]:
gene_pool = set(gene_gene['node1']).union(set(gene_gene['node2']))
print(gene_pool)
print(len(gene_pool))

# TN_edges

In [ ]:
tn_dis_drug = tn_edges[tn_edges['edge'] == 'DIS_DRUG']
tn_dis_drug

In [ ]:
# Disease Filtering
TN_Disease2Drug = tn_dis_drug[tn_dis_drug['node1'].isin(disease_pool)]
TN_Disease2Drug

In [ ]:
# Drug Filtering
TN_Disease2Drug = TN_Disease2Drug[TN_Disease2Drug['node2'].isin(drug_pool_str)]
TN_Disease2Drug

# CSV

- nodes_relation_4type_filtered.csv
- edges_relation_4type_filtered.csv
- TN_edges_relation_4type_filtered.csv

In [ ]:
print(disease_pool)
print(len(disease_pool))
print(gene_pool)
print(len(gene_pool))
print(drug_pool)
print(len(drug_pool))

In [ ]:
# nodes_relation_4type_filtered.csv
all_nodes = disease_pool.union(gene_pool).union(drug_pool)
print(len(all_nodes))
print(len(disease_pool) + len(gene_pool) + len(drug_pool))

nodes_filtered = nodes[nodes['DOID'].isin(all_nodes)]

In [ ]:
with open(r'..\HQ_DIR\graph_files\nodes_relation_4type_filtered.csv', 'w') as f:
    for doid, node_type in zip(nodes_filtered['DOID'], nodes_filtered['Type']):
        f.write(f"{doid}\t{node_type}\n")

In [ ]:
# edges_relation_4type_filtered.csv
all_edges = pd.concat([Drug2Gene,gene_gene,Gene2Disease,Disease2Drug], ignore_index=True)
print(len(all_edges))
print(len(Drug2Gene) + len(gene_gene) + len(Gene2Disease) + len(Disease2Drug))

In [ ]:
with open(r'..\HQ_DIR\graph_files\edges_relation_4type_filtered.csv', 'w') as f:
    for node1,edge,node2,idk1,idk2 in zip(all_edges['node1'], all_edges['edge'], all_edges['node2'], all_edges['idk1'], all_edges['idk2']):
        f.write(f"{node1}\t{edge}\t{node2}\t{idk1}\t{idk2}\n")

In [ ]:
TN_Disease2Drug

In [ ]:
with open(r'..\HQ_DIR\graph_files\TN_edges_relation_4type_filtered.csv', 'w') as f:
    for node1,edge,node2,idk1,idk2 in zip(TN_Disease2Drug['node1'], TN_Disease2Drug['edge'], TN_Disease2Drug['node2'], TN_Disease2Drug['idk1'], TN_Disease2Drug['idk2']):
        f.write(f"{node1}\t{edge}\t{node2}\t{idk1}\t{idk2}\n")